In [1]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *

Importing module 'gym_38' (/data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.1.1
Device count 4
/data2/users/nk523/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data2/users/nk523/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /data2/users/nk523/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


ninja: no work to do.


In [2]:
from env import FrankaPandaEnv
from motion_primitive import MotionPrimitive

In [3]:
def wrap_angle(angle):
    """ Wrap an angle to the range [-pi, pi] """
    return angle - 2 * torch.pi * torch.floor((angle + torch.pi) / (2 * torch.pi))

In [4]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False)

Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: disabled
Loading asset 'urdf/franka_description/robots/franka_panda_gripper.urdf' from 'assets'
Creating 36 environments


In [5]:
motion_primitive = MotionPrimitive(num_envs=env.num_envs)

In [7]:
env.reset_pose("default")
env._refresh()
for _ in range(10000):
    env._refresh()
    dpose = torch.zeros((env.num_envs, 7))
    dpose[:, :3] = torch.Tensor([0.0, 0.2, 0.03]) - env.states["eef_pos"]

    target_quat = torch.Tensor([1, 0, 0, 0])
    current_quat = env.states["eef_quat"]
    diff_quat = quat_mul(
        target_quat.unsqueeze(0).expand_as(current_quat),
        quat_conjugate(current_quat),
    )
    euler_angles = get_euler_xyz(diff_quat)
    dpose[:, 3], dpose[:, 4], dpose[:, 5] = wrap_angle(euler_angles[0]), wrap_angle(euler_angles[1]), wrap_angle(euler_angles[2])
    dpose[:, -1] = -1
    
    actions = motion_primitive.sample_actions()
    dpose[:, :2] = actions * 0.3
    dpose[:, 2] *= 3.0
    env._refresh()
    env.pre_physics_step(dpose)
    env.step_physics()
    env.step_rendering()

KeyboardInterrupt: 